In [25]:
import PyPDF2
import re
import pandas as pd
import os
import matplotlib.pyplot as plt

In [26]:
def generate_text_from_pdf(path):
    convert_text_name=path[:-4]+'.txt'
    os.system('pdftotext -layout %s %s'%(path,convert_text_name))
    return convert_text_name

在fp.readlines()中返回的是个list，每个list中的选项是byte类型的obj,pattern.search的对象必须是str，所以才加上line.decode('utf-8')

In [27]:
def extract_info_from_convert_text(path):
    pattern=re.compile(r'^\d{2}\/\d{2}\/\d{4}.*')
    info_list=[]
    with open(path,'rb') as fp:
        for line in fp.readlines():
            match=pattern.search(line.decode('utf-8'))
            if match:
                info_list.append(line.decode('utf-8'))
    return info_list

In [28]:
print(extract_info_from_convert_text('canberra_land_sales_result.txt'))

['03/02/2017      Residential             3    Throsby       1 to 20        43       8,408            20           $7,899,900             Delta Design and Construction\n', '03/02/2017      Residential             5    Throsby    12;15;17 &        44       4,667            13           $4,810,000             Roman Development Holdings\n', '03/02/2017      Residential             6    Throsby       2 to 25        45       9,865            24           $9,393,600             Delta Design and Construction\n', '03/02/2017      Residential             9    Throsby    3;6;7;9 & 11      35       2,025             5           $1,854,000                     MSL Projects\n', '03/02/2017      Residential         11       Throsby                      33       2,954             7           $2,698,600                     MSL Projects\n', '03/02/2017      Residential         15       Throsby    1-14; 1-15; 1   4; 2; 3    18,101           45           $14,631,150             Wellington Property Group\n

In [29]:
def extract_date_type_suburb_price_company_from_list(info_list):
    for item in info_list:
        date=item[:10]
        pattern_type=re.compile(r'(?:\d{2}\/\d{2}\/\d{4} +)([a-zA-Z]+)')
        block_type=pattern_type.search(item)
        pattern_price=re.compile(r'\$[0-9,]+')
        price=pattern_price.search(item)
        pattern_suburb=re.compile(r'(?<= {3})[a-zA-Z]+(?= {3})')
        
        info_dict={}
        info_dict['date']=date
        if block_type:
            info_dict['block type']=block_type.groups()[0]
            block_type_position=block_type.end()
            suburb=pattern_suburb.search(item[block_type_position+1:])
            if suburb:
                info_dict['suburb']=suburb.group()
            else:
                info_dict['suburb']=''
        else:
            info_dict['block type']=''
            info_dict['suburb']=''
        if price:
            info_dict['price']=price.group()
            purchaser_offset=price.end()
            purchaser=item[purchaser_offset+1:]
            purchaser=purchaser.replace(' ','')
            info_dict['purchaser']=purchaser
        else:
            info_dict['price']=''
            info_dict['purchaser']=item[-10:].replace(' ','')
                      
        yield info_dict

不加pd.set_option('display.expand_frame_repr', False)这句话，print(df)一行会分裂成两行显示

In [37]:
plt.style.use('ggplot')
%matplotlib inline
pd.set_option('display.max_columns',100)
pd.set_option('display.max_colwidth',400)
pd.set_option('display.precision',3)
pd.set_option('display.expand_frame_repr', False)

In [38]:
info_list=extract_info_from_convert_text('canberra_land_sales_result.txt')
land_list=list(extract_date_type_suburb_price_company_from_list(info_list))
df=pd.DataFrame(land_list)

In [39]:
print(df)

      block type        date        price                                      purchaser     suburb
0    Residential  03/02/2017   $7,899,900                   DeltaDesignandConstruction\n    Throsby
1    Residential  03/02/2017   $4,810,000                     RomanDevelopmentHoldings\n    Throsby
2    Residential  03/02/2017   $9,393,600                   DeltaDesignandConstruction\n    Throsby
3    Residential  03/02/2017   $1,854,000                                  MSLProjects\n    Throsby
4    Residential  03/02/2017   $2,698,600                                  MSLProjects\n    Throsby
5    Residential  03/02/2017  $14,631,150                      WellingtonPropertyGroup\n    Throsby
6     Industrial  22/12/2016   $2,800,000                                                      Hume
7      Community  30/11/2016   $2,145,000                ChildcareInvestmentsAusPtyLtd\n  Charnwood
8            Mix  30/11/2016     $635,000                                  AnthonyLieu\n     Amaroo


还是这个函数好,df.to_string()  to force it to show the whole table.

In [42]:
print(df.to_string())

      block type        date         price                                      purchaser      suburb
0    Residential  03/02/2017    $7,899,900                   DeltaDesignandConstruction\n     Throsby
1    Residential  03/02/2017    $4,810,000                     RomanDevelopmentHoldings\n     Throsby
2    Residential  03/02/2017    $9,393,600                   DeltaDesignandConstruction\n     Throsby
3    Residential  03/02/2017    $1,854,000                                  MSLProjects\n     Throsby
4    Residential  03/02/2017    $2,698,600                                  MSLProjects\n     Throsby
5    Residential  03/02/2017   $14,631,150                      WellingtonPropertyGroup\n     Throsby
6     Industrial  22/12/2016    $2,800,000                                                       Hume
7      Community  30/11/2016    $2,145,000                ChildcareInvestmentsAusPtyLtd\n   Charnwood
8            Mix  30/11/2016      $635,000                                  Anthon